<font size=7> Easy IoT Functions

These are functions that I imagine could be added to the [braingeneerspy.iot](https://github.com/braingeneers/braingeneerspy/blob/master/braingeneers/iot/messaging.py) package. The purpose is to make using IoT as easy as possible for simple use cases. Note, `schedule` and `time` packages would also have to be added to `braingeneerspy`. 

**Device**
- `import braingeneers.iot as iot` - user only needs to import one thing. Would they still have to import schedule on `.py`?
- `import schedule`
- `iot.start( device_name, device_type, experiment)` - a single command is added to the bottom of user's `.py` file

**Interface- send commands**
- `import braingeneers.iot as iot`
- `iot.send( device_name, command)` - Sends python script that executed by the listening device.

**Interface- get data**
- `iot.get_schedule( device_name/experiment )` - Gets scheduled commands for a device(s) in a human readable format. I plan to add graphs.
- `iot.get_status`

**Interface- change status of device**
- `iot.shutdown( device_name )` - Stops while loop that runs on listening device. User must us `messaging` to actually delete device. 
- `iot.pause` and `iot_run` - pauses and resumes activity on device. Imagine real world scenario where biologist briefly pauses so they can move organoids.

<font color="orange">

**Note:** For now, I left out the data and logging stuff. Both of these things seem device specific. I might add functions for them later. 
    
    
**Limitations:** Commands running at around 1 second or faster might "skip" tasks. This is because the `while` loop's command `mb.get_device_state` can take up to 1 second. (verified this)

<font color="red">

**To Do**
* figure out topic/device hierarchy and what variables user must type in functions.
* Add `get_schedule` functionality for all devices of same experiment
* add assertion statements to run/pause/shutdown functions
* ?add device type as parameter to commands? with this we can follow iot format, but add additional typing for user.
* ?hide all warnings?
* check that the "send schedule" command works with classes
* ?get logging working?... ?perhaps jsut show last logged command?... `get_last_scheduled_run` or `get_last_ran`
* ?optimize `mb.get_device_state`?
* ?build a function that parses schedule from text?

<font color="red">

**Questions**
* Is state listener code done? I could use that in `start` function instead of `get_device_state`. Is that better?
* I'm implicitly assuming device names are unique across device types (topic==device_name). Is that acceptable?
* there are warning messages. Is that worth hiding?
* does user still have to import `schedule`?
* is `mb.shutdown()` necessary in these cases? 
* is there a more appropriate way to set up the device name/ topic? Where are topics on AWS website?
* should I used `test/` or some other iot hierarchy? This requires I add `device_type` to all functions
* is there a smart way to handle warnings?
* what should we do if device restarts? Should we even worry about this?
* once a user has a `.py` file what is the exact command they should type? ?how to check `.py` is running?

<font color="orange">

**Future Work**
* Graph Schedule(s)...parse schedule from text
* Build generalized package for data


In [12]:
#import braingeneers.iot as iot

**To Do**
* Hide warnings
* Check experiment uuid is in the correct way
* logging
* indexes

# <font color="gray">set up notebook

In [3]:
from braingeneers.iot import messaging
import uuid
import schedule
import time
import warnings

# start

## <font color="green">with Pause and Shutdown 

In [ ]:
"feeding/flow_1"

In [ ]:
flow_1

In [4]:
def start(device_name, device_type, experiment):
    """Create a device and have it start listening for commands. This is intended for simple use cases"""
    warnings.filterwarnings(action='once')                                         # stops same warning from appearing more than once
    mb = messaging.MessageBroker(str(uuid.uuid4))                                  # spin up iot
    
    if device_name not in mb.list_devices_by_type( thingTypeName= device_type):    # check if device already exists
        mb.create_device( device_name= device_name, device_type= device_type)      # if not, create it
    else:                                                                          # otherwise, check device is ok and isn't still running
        assert "status" in mb.get_device_state(device_name), f"{device_name} has corrupted data! Talk to data team."
        assert mb.get_device_state(device_name)["status"]=="shutdown", f"{device_name} already exists and isn't shutdown. Please shutdown with 'iot.shutdown({device_name})'"
    mb.update_device_state( device_name, {"experiment":experiment,"status":"run","schedule":[]} )    # reset state for new run
    
    def respondToCommand(topic: str, message: dict):                               # build function that runs when device receives command
        exec(message["command"])                                                   # run python command that was sent
        schedule_str= [f"Job {i}: "+x.__repr__() for i,x in enumerate(schedule.get_jobs())]  # turn schedule into list of strings
        mb.update_device_state( device_name, {"schedule":schedule_str} )           # in case schedule changed, update state's schedule
    mb.subscribe_message( f"devices/{device_type}/{device_name}", respondToCommand )   # start listening for new commands
    
    status = "run"                                                                 # keep python running so that listener can do it's job
    while not status=="shutdown":                                                  # when it's time to stop, iot makes status=shutdown
        if status=="run":                                                          # if the device is in run mode,
            schedule.run_pending()                                                 # run any scheduled commands if it's their time
        status = mb.get_device_state( device_name )["status"]                      # check if we've been told to stop running
        time.sleep(.1)                                                             # wait a little to save cpu usage
    mb.shutdown()                                                                  # shutdown iot at the end.
        

## Original Working Code -- no pause or shutdown

### Try: Schedule reset <font color="red"> -- think about this more

how would I actually bring the schedule back onto the device?

# send

In [ ]:
send( 'feeding/flow_1' , "print('hello')")

In [ ]:
devices/feeding/flow_1
devices/feeding/flow_2

In [5]:
def send( device_name_and_topic, command ):
    """Send a python script as a string which is then implemented by an IoT device. This is intended for simple use cases"""
    mb = messaging.MessageBroker(str(uuid.uuid4))                             # spin up iot
    mb.publish_message( topic=f"devices/{device_name_and_topic}", message={"command": command } )    # send command to listening device
    mb.shutdown()                                                             # shutdown iot

In [ ]:
mb.get_device_state("")

# get_schedule

In [6]:
def get_schedule( device_name ):
    """Get a list of scheduled commands from a device's shadow. This is intended for simple use cases"""
    mb = messaging.MessageBroker(str(uuid.uuid4))                             # spin up iot
    my_schedule = mb.get_device_state( device_name )["schedule"]              # get schedule for device
    mb.shutdown()                                                             # shutdown iot
    return my_schedule                                                        # return schedule to user

# get_status

In [7]:
def get_status( device_name ):
    """Get a list of scheduled commands from a device's shadow. This is intended for simple use cases"""
    mb = messaging.MessageBroker(str(uuid.uuid4))                        # spin up iot
    status = mb.get_device_state( device_name )["status"]                # get schedule for device
    mb.shutdown()                                                        # shutdown iot
    return status                                                        # return schedule to user

# shutdown

In [8]:
def shutdown( device_name ):
    """Stops iot listener on device by changing flag on shadow. This is intended for simple use cases"""
    mb = messaging.MessageBroker(str(uuid.uuid4))                             # spin up iot
    mb.update_device_state( device_name, {"status":"shutdown"} )              # change status flag on device state to shutdown
    mb.shutdown()                                                             # shutdown iot

## Original working code - no pause or shutdown

# pause

In [9]:
def pause( device_name ):
    """Pauses iot listener on device by changing flag on shadow. This is intended for simple use cases"""
    mb = messaging.MessageBroker(str(uuid.uuid4))                             # spin up iot
    mb.update_device_state( device_name, {"status":"pause"} )              # change status flag on device state to pause
    mb.shutdown()        

# run

In [10]:
def run( device_name ):
    """Resumes running of iot listener on device by changing flag on shadow. This is intended for simple use cases"""
    mb = messaging.MessageBroker(str(uuid.uuid4))                        # spin up iot
    mb.update_device_state( device_name, {"status":"run"} )              # change status flag on device state to run
    mb.shutdown()   